In [2]:
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS,cross_origin
import logging
import os
import requests
import re
import pandas as pd

In [3]:
input_text = input("input youtube channel name")
url = f'https://www.youtube.com/@{input_text}/videos'
headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0',
'Accept-Language': 'en-US,en;q=0.9'
}


response = requests.get(url, headers=headers)
response_text = response.text



vid_titles = re.findall('"title":{"runs":\[{"text":".*?"', response_text)

In [4]:
vid_titles

['"title":{"runs":[{"text":"MATTER IN OUR SURROUNDINGS in 1 Shot | FULL Chapter Coverage (Concepts + PYQs) | Class-9th Science"',
 '"title":{"runs":[{"text":"POLYNOMIALS in 1 Shot | FULL Chapter Coverage (Concepts + PYQs) | Class 9th Maths"',
 '"title":{"runs":[{"text":"NAZISM AND THE RISE OF HITLER in 1 Shot || FULL Chapter Coverage (THEORY+PYQs) || Class 9th Sst"',
 '"title":{"runs":[{"text":"NUMBER SYSTEM in 1 Shot || FULL Chapter Coverage (Concepts+PYQs) || Class 9th Maths"',
 '"title":{"runs":[{"text":"SOUND OF MUSIC in 1 Shot || FULL Chapter Coverage (THEORY+PYQs) || Class 9th English"',
 '"title":{"runs":[{"text":"PHYSICAL FEATURES OF INDIA in 1 Shot || FULL Chapter Coverage (THEORY+PYQs) || Class-9th SST"',
 '"title":{"runs":[{"text":"MOTION in 1 Shot || FULL Chapter Coverage (Concepts+PYQs) || Class-9th Science"',
 '"title":{"runs":[{"text":"THE FUN THEY HAD + THE ROAD N0T TAKEN in 1 Shot || FULL Chapter (THEORY+PYQs) || Class - 9th English"',
 '"title":{"runs":[{"text":"CARBO

In [5]:
vid_thumbnails = re.findall(r"https://i.ytimg.com/vi/[A-Za-z0-9_-]{11}/[A-Za-z0-9_]{9}.jpg", response_text)
vid_thumbnails

['https://i.ytimg.com/vi/vawU6R8MaO0/hqdefault.jpg',
 'https://i.ytimg.com/vi/vawU6R8MaO0/hqdefault.jpg',
 'https://i.ytimg.com/vi/vawU6R8MaO0/hqdefault.jpg',
 'https://i.ytimg.com/vi/vawU6R8MaO0/hqdefault.jpg',
 'https://i.ytimg.com/vi/roFOxpZtiV4/hqdefault.jpg',
 'https://i.ytimg.com/vi/roFOxpZtiV4/hqdefault.jpg',
 'https://i.ytimg.com/vi/roFOxpZtiV4/hqdefault.jpg',
 'https://i.ytimg.com/vi/roFOxpZtiV4/hqdefault.jpg',
 'https://i.ytimg.com/vi/Kga4KUerm3o/hqdefault.jpg',
 'https://i.ytimg.com/vi/Kga4KUerm3o/hqdefault.jpg',
 'https://i.ytimg.com/vi/Kga4KUerm3o/hqdefault.jpg',
 'https://i.ytimg.com/vi/Kga4KUerm3o/hqdefault.jpg',
 'https://i.ytimg.com/vi/xn2HskGqSkI/hqdefault.jpg',
 'https://i.ytimg.com/vi/xn2HskGqSkI/hqdefault.jpg',
 'https://i.ytimg.com/vi/xn2HskGqSkI/hqdefault.jpg',
 'https://i.ytimg.com/vi/xn2HskGqSkI/hqdefault.jpg',
 'https://i.ytimg.com/vi/JoBEnYLkMDE/hqdefault.jpg',
 'https://i.ytimg.com/vi/JoBEnYLkMDE/hqdefault.jpg',
 'https://i.ytimg.com/vi/JoBEnYLkMDE/hqdefault

In [6]:
vid_links = re.findall(r"watch\?v=[A-Za-z0-9_-]{11}", response_text)
vid_links

['watch?v=vawU6R8MaO0',
 'watch?v=roFOxpZtiV4',
 'watch?v=Kga4KUerm3o',
 'watch?v=xn2HskGqSkI',
 'watch?v=JoBEnYLkMDE',
 'watch?v=NcnFp3gh_u8',
 'watch?v=Msy44HhRGRw',
 'watch?v=hyWXw83I0pc',
 'watch?v=kv96sQulwZc',
 'watch?v=dxD8krpKP-8',
 'watch?v=Z0xlPJtojLs',
 'watch?v=UO2QcKq-jv0',
 'watch?v=DEDdCk1gBRk',
 'watch?v=0mEBbPkb2eQ',
 'watch?v=YKrmQkDhvjA',
 'watch?v=Zuk6hSgdsQI',
 'watch?v=V_xsifaP1m4',
 'watch?v=OA3RgocBJHI',
 'watch?v=zJyx_KYmg18',
 'watch?v=v8znciuJuew',
 'watch?v=tMyLWMxnCus',
 'watch?v=PujmnI1ncjc',
 'watch?v=C5-ipvDYEdE',
 'watch?v=5QNlFSJ7yo0',
 'watch?v=yW28jdDolug',
 'watch?v=IVIL5blWJps',
 'watch?v=X__dNCg7OsA',
 'watch?v=zRrPJxMT8R0',
 'watch?v=_yd3jVElO_g',
 'watch?v=WuerM0oWIqk']

In [7]:
pattern3 = re.compile(r"[0-9]+(\.[0-9]+)?[a-zA-Z]*K views")  # view count
pattern3

re.compile(r'[0-9]+(\.[0-9]+)?[a-zA-Z]*K views', re.UNICODE)

In [8]:
pattern4 = re.compile(r"\d+ (minutes|hours|hour|days|day|weeks|week|years|year) ago")  # vedio age
pattern4

re.compile(r'\d+ (minutes|hours|hour|days|day|weeks|week|years|year) ago',
re.UNICODE)

In [9]:
matches1 = pattern3.finditer(response_text)
matches2 = pattern4.finditer(response_text)

In [10]:
vid_viewcounts=[]
vid_ages=[]
count = 0
for match1,match2 in zip(matches1,matches2):
    vid_ages.append(match2[0])
    vid_viewcounts.append(match1[0])

In [11]:
titles = vid_titles[0:10]
thumbnails = list(dict.fromkeys(vid_thumbnails))
links = vid_links[0:10]
viewcounts=vid_viewcounts[0:20:2]
ages=vid_ages[0:20:2]

details_list=[]

for title,thumbnail,link,viewcount,age in zip(titles,thumbnails,links,viewcounts,ages):
    details_dict={
    "title":title.split('"')[-2], "thumbnail": thumbnail, "link": "https://www.youtube.com/"+link,
    "viewcount": viewcount, "age": age
    }
    details_list.append(details_dict)



details_list

[{'title': 'MATTER IN OUR SURROUNDINGS in 1 Shot | FULL Chapter Coverage (Concepts + PYQs) | Class-9th Science',
  'thumbnail': 'https://i.ytimg.com/vi/vawU6R8MaO0/hqdefault.jpg',
  'link': 'https://www.youtube.com/watch?v=vawU6R8MaO0',
  'viewcount': '11K views',
  'age': '1 day ago'},
 {'title': 'POLYNOMIALS in 1 Shot | FULL Chapter Coverage (Concepts + PYQs) | Class 9th Maths',
  'thumbnail': 'https://i.ytimg.com/vi/roFOxpZtiV4/hqdefault.jpg',
  'link': 'https://www.youtube.com/watch?v=roFOxpZtiV4',
  'viewcount': '8.3K views',
  'age': '1 day ago'},
 {'title': 'NAZISM AND THE RISE OF HITLER in 1 Shot || FULL Chapter Coverage (THEORY+PYQs) || Class 9th Sst',
  'thumbnail': 'https://i.ytimg.com/vi/Kga4KUerm3o/hqdefault.jpg',
  'link': 'https://www.youtube.com/watch?v=Kga4KUerm3o',
  'viewcount': '11K views',
  'age': '2 days ago'},
 {'title': 'NUMBER SYSTEM in 1 Shot || FULL Chapter Coverage (Concepts+PYQs) || Class 9th Maths',
  'thumbnail': 'https://i.ytimg.com/vi/xn2HskGqSkI/hqdef

In [13]:
df = pd.DataFrame(details_list)
df.to_csv('YT_Data.csv', index=False)